**<font color = black size=6>实验六:随机森林</font>**

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from collections import Counter

**<font color = blue size=4>第一部分:函数介绍</font>**

介绍一些可能会用到的函数。

In [34]:
# np.random.choice函数从一个一维数组中随机采样
x = np.array([1,2,3,4])
y = np.random.choice(x, replace=True, size=10)
print(y)

# np.random.shuffle函数对一个数组/矩阵按照第一维进行洗牌
x = np.array([[0,1,2],[3,4,5],[6,7,8],[9,10,11],[12,13,14]])
np.random.shuffle(x)
print(x)

# DataFrame对象的sample函数可以随机采样n个数据或者采样比例为frac的数据
x = np.array([[0,0,0],[1,1,1],[2,2,2],[3,3,3],[4,4,4]])
frame = pd.DataFrame(x)
print(frame.sample(n=2))
print(frame.sample(frac=0.3))

[3 4 2 1 1 3 4 4 1 3]
[[ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [ 0  1  2]]
   0  1  2
2  2  2  2
0  0  0  0
   0  1  2
4  4  4  4
1  1  1  1


**<font color = blue size=4>第二部分:实验任务</font>**

本次实验承接上次实验，实现随机森林。

<span style="color:white">本次实验依旧使用泰坦尼克号数据集(train_titanic.csv, test_titanic.csv。数据集包括了四个属性特征以及一个标签(即为Survived,代表是否生还),属性特征分别为Sex(性别)，sibsp(堂兄弟妹个数)，Parch(父母与小孩的个数)，Pclass(乘客等级)  
其中该数据集无缺失值和异常值，且所有连续变量已自动转换为离散变量,标签(Survived)也自动转变为离散变量0和1，所以你无需进行数据预处理，可以直接使用该数据集。</span>

<span style="color:white">1) 对上次实验的best_split函数进行修改，实现随机特征选择。  
先从特征集$A$中先随机选取$k$个特征构成特征集$A'$，再从$A'$中选取最佳划分的特征。$k$一般取$max\{log_2 d,1\}$, $d$是$A$的元素的个数。你可使用特征的信息增益来决定最佳划分的特征。  
    【输入】：数据集D、特征集A    
    【输出】：随机特征集A'中最佳划分的特征维数   
    【信息增益公式】:  
        某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征feature,该特征包含V个不同的取值，特征值为第v(v=1,2,...,V)个值的数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$  
    【信息熵公式】:  
        某数组包含K个不同的取值，样本为第k(k=1,2,...,K)个值的数量所占比例为p_k,则其信息熵为$$Ent=-\sum_{k=1}^K p_k log_2 p_k$$
</span>

In [35]:
def entropy(label):
    ent = 0
    # 统计
    c = Counter(label)
    num = c.values()
    p = np.array(list(num))/sum(num)
    # print(p)
    # calculate entropy
    ent = -(p*np.log2(p)).sum()
    return ent

def split(feature, label, d):
    # print(feature)
    # print(label)
    split_feature = []
    split_label = []
    
    # get feature of the dim
    feature_specified_dim = feature[feature.columns[d]]
    # print(feature_specified_dim)
    c = Counter(feature_specified_dim)
    # print(c)

    for key in c.keys():
        key = int(key)
        _index = [i for (i, v) in enumerate(feature_specified_dim)
                 if key==v]
        split_feature.append(feature.iloc[_index])
        split_label.append(label.iloc[_index])
    
    # print('split_feature\n',split_feature)
    # print('split_label\n',split_label)
    return split_feature,split_label

def best_split(D:pd.DataFrame, A):
    
    label = D['Survived']
    feature = D.drop(['Survived'],axis=1)
    
    # init
    best_entropy = 0
    best_dimension = -1
    # calculate ent
    without_ent = entropy(label)
    
    # select random k feature as A'
    k = max(int(math.log2(len(A))),1)
    assert isinstance(k,int)
    assert k>=1
    
    A_k = random.sample(A,k)
    
    # iter for each feature
    for dim in A_k:
        split_feature,split_label = split(feature,label,dim)
        with_ent = 0
        # each group
        for group_label in split_label:
            
            _ent = entropy(group_label)
            # print('_ent',_ent)
            
            percent = len(group_label)/len(label)
            # print('percent',percent)
            
            with_ent += percent*_ent
        
        # calculate the max entropy add
        if(without_ent-with_ent>best_entropy):
            best_entropy = without_ent-with_ent
            best_dimension = dim
            
    if(len(A_k)==1):
        return A_k[0]

    return best_dimension

<span style="color:white">2) 对上次实验完成的决策树类进行修改。你需要实现下面三个函数：  
1. TreeGenerate(self, D, A)：递归构建决策树，伪代码参照提供的“Algorithm 1 决策树学习基本算法”。  
2. train(self, D)：做一些数据预处理，包括将Dataframe转换为numpy矩阵，从数据集中提取属性集，并调用TreeGenerate函数来递归地生成决策树。  
3. predict(self, D)：对测试集D进行预测，要求返回数据集D的预测标签，即一个(|D|,1)矩阵（|D|行1列）。  
由于训练集是采样生成，因此需要对predict函数做修改。需要考虑测试集中出现决策树无法划分的特征值时的情况。给出两种参考的做法：  
a).对其不再进行预测，直接给定划分失败的样本标签(例如-1)。  
b).跳过该划分节点，随机选取一个特征值继续遍历。</span>

In [36]:
# 记下所有属性可能的取值
train_frame = pd.read_csv('train_titanic.csv')
D = np.array(train_frame)
A = set(range(D.shape[1] - 1))
possible_value = {}
for every in A:
    possible_value[every] = np.unique(D[:, every])

In [37]:
# 树结点类
class Node:
    def __init__(self, isLeaf=True, label=-1, index=-1):
        self.isLeaf = isLeaf # isLeaf表示该结点是否是叶结点
        self.label = label # label表示该叶结点的label（当结点为叶结点时有用）
        self.index = index # index表示该分支结点的划分属性的序号（当结点为分支结点时有用）
        self.children = {} # children表示该结点的所有孩子结点，dict类型，方便进行决策树的搜索
        
    def addNode(self, val, node):
        self.children[val] = node #为当前结点增加一个划分属性的值为val的孩子结点

In [38]:
# 决策树类
class DTree:
    def __init__(self):
        self.tree_root = None #决策树的根结点
        self.possible_value = {} # 用于存储每个特征可能的取值
    
    '''
    TreeGenerate函数用于递归构建决策树，伪代码参照课件中的“Algorithm 1 决策树学习基本算法”
     
    '''    
    
    def TreeGenerate(self, D, A:set):
        
        # 生成结点 node
        node = Node()

        
        # if D中样本全属于同一类别C then
        #     将node标记为C类叶结点并返回
        # end if
        # print('A',A)
        # print('possible_value',self.possible_value)
        label = D['Survived']
        # print('label',label)
        # belong to 1 class
        if(len(np.unique(label))==1):
            node.isLeaf = True
            leaf_label = np.unique(label)[0]
            # print('leaf_label',leaf_label)
            node.label = leaf_label
            return node
        
        
        # if A = Ø OR D中样本在A上取值相同 then
        #     将node标记叶结点，其类别标记为D中样本数最多的类并返回
        # end if
        A_null=False
        A_same = False
        if(len(A)==0):
            A_null=True
        else:
            count = 0
            for feature_index in A:
                feature = D[D.columns[feature_index]]
                # print('feature',feature)   
                c = Counter(feature)   
                # print('len(c.keys())',len(c.keys()))      
                if(len(c.keys())==1):
                    count+=1
            if(count==len(A)):
                A_same=True
        # judge
        if(A_same or A_null):
            node.isLeaf=True 
            c = Counter(label)
            leaf_label = c.most_common(1)[0][0]
            # print('leaf_label',leaf_label)
            node.label = leaf_label
            return node
        
        
        # 从A中选择最优划分特征a_star
        # （选择信息增益最大的特征，用到上面实现的best_split函数） 
        best_dim = best_split(D,A)
        # print('best_dim',best_dim)
        
        
        
        # for a_star 的每一个值a_star_v do
        #     为node 生成每一个分支；令D_v表示D中在a_star上取值为a_star_v的样本子集
        #     if D_v 为空 then
        #         将分支结点标记为叶结点，其类别标记为D中样本最多的类
        #     else
        #         以TreeGenerate(D_v,A-{a_star}) 为分支结点
        #     end if
        # end for
        a_star = self.possible_value[best_dim]
        # print('a_star',a_star)    
        for a_star_v in a_star:
            child = Node()
            name = D.columns[best_dim]
            # print('name',name)
            _D = D[D[name]==a_star_v]
            # print('_D',_D)
            if(len(_D)==0):
                child.isLeaf=True
                c = Counter(label)
                leaf_label = c.most_common(1)[0][0]
                child.label=leaf_label
            else:
                # print('origin_A',A)
                new_A = A.copy()
                new_A.remove(best_dim)
                # print('new_A',new_A)
                child = self.TreeGenerate(_D, new_A)
                
            node.isLeaf=False
            node.feature_index=best_dim
            node.addNode(a_star_v,child)
            
        return node
    
    
 
    '''
    train函数可以做一些数据预处理（比如Dataframe到numpy矩阵的转换，提取属性集等），并调用TreeGenerate函数来递归地生成决策树
 
    '''
    
    def train(self, D:pd.DataFrame):
#         D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        A = set(range(D.shape[1] - 1)) # 特征集A
        
        #记下每个特征可能的取值
        for every in A:
            self.possible_value[every] = np.unique(np.array(D.iloc[:, every]))
        
        self.tree_root = self.TreeGenerate(D, A) # 递归地生成决策树，并将决策树的根结点赋值给self.tree_root
        return

    
    '''
    predict函数对测试集D进行预测， 并输出预测准确率（预测正确的个数 / 总数据数量）
    
    '''
    
    def predict(self, D:pd.DataFrame):
        
#         D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）      
#         #对于D中的每一行数据d，从当前结点x=self.tree_root开始，当当前结点x为分支结点时，

#         #则搜索x的划分特征为该行数据相应的特征值的孩子结点（即x=x.children[d[x.index]]），不断重复，
#         #直至搜索到叶结点，该叶结点的标签就是数据d的预测标签
        root = self.tree_root 
        
        num = D.shape[0]
        right_num = 0
        # 预测结果
        pred = D.copy()
        _pred = np.array([])
        # iter
        for i in range(num):
            # back to root node!!!!!!!!
            node = root
            
            data_line = D.iloc[i,:]
            # print('data_line',data_line)
            
            # visit tree
            assert isinstance(data_line, pd.Series)
            while(not node.isLeaf):
                dim = node.feature_index
                value = data_line[dim]
                # print('value',value)
                # print('分支节点，根据dim=',dim,',value=',value,'进入子节点\n')
                # judge if test[dim] in node.children's keys
                if value in node.children: 
                    node = node.children[value]
                else:
                    # random select a key in node.children
                    random_key = random.choice(list(node.children.keys()))
                    node = node.children[random_key]
            # to leaf
            # print('叶子节点\n')
            predict_label = node.label
            # print('predict label',predict_label)
            label = data_line['Survived']
            # print('true label',label)
            
            assert predict_label is not None
            _pred = np.concatenate((_pred,[predict_label]))
            if(label==predict_label):
               right_num += 1 
               
               
            

        # print('-----pred------\n',_pred)
        pred['predict'] = _pred
        acc = right_num/num
        return pred, acc


<span style="color:white">3) 重复采用Bootstrap自助采样法对训练数据集'test_titanic.csv'进行采样，生成$n$个子训练数据集($n$自行设定)。  
Bootstrap采样法是指，每次从原数据集中【有放回】地随机采样一个样本，重复进行$m$次，就生成一个有$m$个样本的子数据集。</span>

In [39]:
train_frame = pd.read_csv('train_titanic.csv')

# Bootstrap 采样
n = 10# 采样10次
m = train_frame.shape[0]
samples = {}

for i in range(n):
    sample = pd.DataFrame()
    indexs = np.array(train_frame.index.values)
    print('indexs',indexs)
    sample_indexs = np.random.choice(indexs, replace=True, size=m)
    print('sample_indexs',sample_indexs)
    print('sample_indexs_len',len(sample_indexs))
    sample = train_frame.iloc[sample_indexs]
    print('sample_len',len(sample))
    samples[i] = sample
    # test
    # test = sample.copy()
    # test1 = test.drop_duplicates()
    # print(test1.shape[0])
    # print('test1',test1)
    # print(test.shape[0])
    
    


indexs [   0    1    2 ... 1006 1007 1008]
sample_indexs [500 970 470 ... 475 144 944]
sample_indexs_len 1009
sample_len 1009
indexs [   0    1    2 ... 1006 1007 1008]
sample_indexs [467 695 120 ... 683  81 375]
sample_indexs_len 1009
sample_len 1009
indexs [   0    1    2 ... 1006 1007 1008]
sample_indexs [785 701 858 ... 730 855 161]
sample_indexs_len 1009
sample_len 1009
indexs [   0    1    2 ... 1006 1007 1008]
sample_indexs [ 87  51 153 ... 513 333  23]
sample_indexs_len 1009
sample_len 1009
indexs [   0    1    2 ... 1006 1007 1008]
sample_indexs [602 600 859 ... 421 610 748]
sample_indexs_len 1009
sample_len 1009
indexs [   0    1    2 ... 1006 1007 1008]
sample_indexs [178 196 641 ... 331  43 300]
sample_indexs_len 1009
sample_len 1009
indexs [   0    1    2 ... 1006 1007 1008]
sample_indexs [ 544   21  217 ...  758  786 1000]
sample_indexs_len 1009
sample_len 1009
indexs [   0    1    2 ... 1006 1007 1008]
sample_indexs [903 660 394 ... 440 485 280]
sample_indexs_len 1009
sa

<span style="color:white">4) 生成n棵决策树实例，使用上述生成的n个子训练数据集各自训练一棵决策树，即子训练集D1训练决策树1，子训练集D2训练决策树2……</span>

In [40]:
# train n trees
trees = {}

for index in range(n):
    train_df = samples[index]
    # 构建决策树
    tree = DTree()
    tree.train(train_df)
    trees[index] = tree
    


C:\Users\37359\AppData\Local\Temp\ipykernel_3500\3482167606.py:51: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  A_k = random.sample(A,k)
C:\Users\37359\AppData\Local\Temp\ipykernel_3500\3482167606.py:51: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  A_k = random.sample(A,k)
C:\Users\37359\AppData\Local\Temp\ipykernel_3500\3482167606.py:51: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  A_k = random.sample(A,k)
C:\Users\37359\AppData\Local\Temp\ipykernel_3500\3482167606.py:51: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  A_k = random.sample(A,k)
C:\Users\37359\AppData\Local\Temp\ipykernel_3500\3482167606.py:51: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent v

<span style="color:white">5) 用训练完成的$n$棵决策树分别对测试数据集'test_titanic.csv'进行预测。采用相对多数投票法来对各棵决策树的预测结果进行结合。输出结合的预测结果的准确率。  
【相对多数投票法】  
对于某个样本$x$, 相对多数投票法预测它的标签为得票最多的标签。若同时有多个标签获得最高票，则从中随机选取一个。其公式如下所示：
$$H(x)=C_{\mathop{\arg\max}_{j} \sum_{i=1}^n h_i^j(x)}$$  
</span>

In [44]:
test_frame = pd.read_csv('test_titanic.csv')
test_num = test_frame.shape[0]
# predict

predicts = {}
for i in range(n):
    tree:DTree = trees[i]
    pred,single_acc = tree.predict(test_frame)
    print('第',i,'棵树的acc rate',single_acc)
    predicts[i] = pred
    
# vote
final_pred = pd.DataFrame({'final pred': np.ones(test_num,dtype=int)})

# each test data line 
for i in range(test_num):
    
    results = []
    # n predict result
    for j in range(n):
        predict = predicts[j]
        # print('--------predict\n',predict)
        _pred_each = predict.iloc[i,5]
        # print('_pred_each',_pred_each)
        results.append(int(_pred_each))
        
    c = Counter(results)
    # print(c)
    # print('results\n',results)
    final = c.most_common()[0][0]
    # print('final',final)
    final_pred.iloc[i,0] = final

# print(final_pred)

# get final acc rate
# print('final_pred',final_pred[:20])
test_frame['result'] = final_pred
test_frame['is_right'] = test_frame['result'] == test_frame['Survived']
print('test_frame',test_frame[:20])
print('final acc rate:',np.sum(test_frame['is_right'])/test_frame.shape[0])



第 0 棵树的acc rate 0.8465346534653465
第 1 棵树的acc rate 0.8267326732673267
第 2 棵树的acc rate 0.7821782178217822
第 3 棵树的acc rate 0.8316831683168316
第 4 棵树的acc rate 0.8118811881188119
第 5 棵树的acc rate 0.8118811881188119
第 6 棵树的acc rate 0.8267326732673267
第 7 棵树的acc rate 0.8118811881188119
第 8 棵树的acc rate 0.8267326732673267
第 9 棵树的acc rate 0.8168316831683168
test_frame     Sex  sibsp  Parch  Pclass  Survived  result  is_right
0     0      0      0       1         0       0      True
1     1      1      0       1         1       1      True
2     1      0      0       2         0       1     False
3     1      1      0       1         1       1      True
4     1      1      1       2         0       1     False
5     0      0      0       2         0       0      True
6     0      0      0       3         0       0      True
7     0      0      0       2         0       0      True
8     0      1      2       2         0       0      True
9     1      0      0       2         0       1     False
1

In [43]:
#展示生成的决策树结构
def display_tree(node, indent=''):
    if node.isLeaf:
        print(indent + "Leaf Node: label =", node.label)
    else:
        print(indent + "Branch Node: feature_index =", node.feature_index)
        for value, child in node.children.items():
            print(indent + "|--> Child Value:", value)
            display_tree(child, indent + "   ")


display_tree(trees[0].tree_root)


Branch Node: feature_index = 0
|--> Child Value: 0
   Branch Node: feature_index = 1
   |--> Child Value: 0
      Branch Node: feature_index = 2
      |--> Child Value: 0
         Branch Node: feature_index = 3
         |--> Child Value: 1
            Leaf Node: label = 0
         |--> Child Value: 2
            Leaf Node: label = 0
         |--> Child Value: 3
            Leaf Node: label = 0
      |--> Child Value: 1
         Branch Node: feature_index = 3
         |--> Child Value: 1
            Leaf Node: label = 0
         |--> Child Value: 2
            Leaf Node: label = 0
         |--> Child Value: 3
            Leaf Node: label = 0
      |--> Child Value: 2
         Branch Node: feature_index = 3
         |--> Child Value: 1
            Leaf Node: label = 1
         |--> Child Value: 2
            Leaf Node: label = 0
         |--> Child Value: 3
            Leaf Node: label = 0
      |--> Child Value: 3
         Leaf Node: label = 0
      |--> Child Value: 4
         Leaf Nod

**<font color = blue size=4>第三部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)不要提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可

二、实验报告提交截止日期为：【10月27日 14:20】  
提交地址：https://send2me.cn/g_kfMtFI/SuiqyPO6B7rxqg  
要求：  
1)文件格式为：学号-姓名-实验六.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  

三、课堂课件获取地址: https://www.jianguoyun.com/p/DTGgCYAQp5WhChir26EFIAA  
实验内容获取地址: https://www.jianguoyun.com/p/DekWAFoQp5WhChis26EFIAA